<a href="https://colab.research.google.com/github/mohammedidriss/Group-9---small-micro-and-nano-language-Models/blob/main/SLM_Fine-tuning_with_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  Install Libraries
print("Installing necessary libraries...")
!pip install -q transformers datasets peft accelerate bitsandbytes trl torch wikipedia evaluate rouge_score bert_score sentence-transformers faiss-cpu

print("\nEnsuring 'bitsandbytes' is up-to-date...")
!pip install -U bitsandbytes

# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')
print("\n✅ Google Drive mounted successfully.")

# import libraries
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup
)
from transformers.trainer_utils import get_last_checkpoint
from datasets import load_dataset
import torch.nn.functional as F
import torch.nn as nn
import os
import wikipedia
import time
import evaluate
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import json

print("✅ All libraries imported.")



Installing necessary libraries...

Ensuring 'bitsandbytes' is up-to-date...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

✅ Google Drive mounted successfully.
✅ All libraries imported.


In [ ]:
# model training

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from transformers.trainer_utils import get_last_checkpoint
from datasets import load_dataset
import os

# shared configuration
class ProjectConfig:
    DRIVE_PROJECT_DIR = "/content/drive/MyDrive/SLM_Project"
    # save-the new model will be saved to a different directory to avoid confusion
    STUDENT_MODEL_OUTPUT_DIR = os.path.join(DRIVE_PROJECT_DIR, "distilgpt2_squad_fine_tuned")

    # Training-specific parameters
    BASE_MODEL_ID = "distilgpt2"
    #  Use a dataset designed for contextual Q&A
    DATASET_ID = "squad"
    MAX_TOKEN_LENGTH = 1024 # SQuAD requires a larger context window
    NUM_TRAIN_EPOCHS = 3
    BATCH_SIZE = 2
    GRADIENT_ACCUMULATION_STEPS = 8
    LEARNING_RATE = 5e-5
    OUTPUT_DIR = "./training_output_squad_finetune"

config = ProjectConfig()

os.makedirs(config.STUDENT_MODEL_OUTPUT_DIR, exist_ok=True)
print(f"Model will be saved to: {config.STUDENT_MODEL_OUTPUT_DIR}")

# load base model and tokenizers
print("\nLoading base model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(config.BASE_MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(config.BASE_MODEL_ID)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model.to(DEVICE)
print(f"Base model ('{config.BASE_MODEL_ID}') loaded with {model.num_parameters():,} parameters.")

# prepare and lod squad dataset for training
print("\nPreparing the SQuAD dataset...")
dataset = load_dataset(config.DATASET_ID, split='train')

def format_and_tokenize_squad(examples):
    # The RAG prompt template we will use during inference
    rag_template = "Context:\n{context}\n\nInstruction:\n{question}\n\nResponse:\n{answer}"

    formatted_texts = []
    for i in range(len(examples["question"])):
        # SQuAD answers are in a nested dictionary
        answer_text = examples["answers"][i]["text"][0]

        text = rag_template.format(
            context=examples["context"][i],
            question=examples["question"][i],
            answer=answer_text
        )
        # add the eos_token to signal the end of a sequence
        formatted_texts.append(text + tokenizer.eos_token)

    return tokenizer(
        formatted_texts,
        padding="max_length",
        truncation=True,
        max_length=config.MAX_TOKEN_LENGTH,
    )

tokenized_dataset = dataset.map(
    format_and_tokenize_squad, batched=True, remove_columns=dataset.column_names
)
print("Dataset prepared and formatted for RAG-style training.")

# the actual training job
print("\nStarting RAG-aware fine-tuning run...")

training_args = TrainingArguments(
    output_dir=config.OUTPUT_DIR,
    num_train_epochs=config.NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=config.BATCH_SIZE,
    gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
    learning_rate=config.LEARNING_RATE,
    logging_steps=100,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    report_to="none",
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

last_checkpoint = get_last_checkpoint(config.OUTPUT_DIR)
if last_checkpoint is not None:
    print(f"✅ Checkpoint found at {last_checkpoint}. Resuming training.")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("🚀 No checkpoint found. Starting a fresh training run.")
    trainer.train()

print("Training finished.")

# save model to google drive
print(f"\nSaving the final, RAG-aware model to Google Drive: {config.STUDENT_MODEL_OUTPUT_DIR}...")
unwrapped_model = trainer.model
unwrapped_model.save_pretrained(config.STUDENT_MODEL_OUTPUT_DIR)
tokenizer.save_pretrained(config.STUDENT_MODEL_OUTPUT_DIR)
print("✅ Model saved successfully to Google Drive.")


Model will be saved to: /content/drive/MyDrive/SLM_Project/distilgpt2_squad_fine_tuned

Loading base model and tokenizer...
Base model ('distilgpt2') loaded with 81,912,576 parameters.

Preparing the SQuAD dataset...


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

/tmp/ipython-input-2595522793.py:106: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.


Dataset prepared and formatted for RAG-style training.

Starting RAG-aware fine-tuning run...
🚀 No checkpoint found. Starting a fresh training run.


Step,Training Loss
100,3.414700
200,3.330500
300,3.311300
400,3.289600
500,3.274900
600,3.252000
700,3.253400
800,3.239500
900,3.209300
1000,3.209700


Training finished.

Saving the final, RAG-aware model to Google Drive: /content/drive/MyDrive/SLM_Project/distilgpt2_squad_fine_tuned...
✅ Model saved successfully to Google Drive.


In [ ]:
#build and save the vector databse
import os
import json
import torch
import numpy as np
import faiss
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

print("\n--- Starting Vector Database Creation ---")

# configuration
class ProjectConfig:
    DRIVE_PROJECT_DIR = "/content/drive/MyDrive/SLM_Project"
    VECTOR_DB_PATH = os.path.join(DRIVE_PROJECT_DIR, "faiss_wikipedia_index.bin")
    KNOWLEDGE_BASE_PATH = os.path.join(DRIVE_PROJECT_DIR, "wikipedia_chunks.json")

config = ProjectConfig()

# text chunk function
def chunk_text(text, chunk_size=256, overlap=20):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

# load the knowledge base-first
print("Loading official Wikipedia dataset (this may take a moment)...")
try:
    # Use tthe official Wikimedia dataset and stream it to save memory.
    wiki_dataset_stream = load_dataset("wikimedia/wikipedia", "20231101.en", split='train', streaming=True)

    knowledge_base_texts = []
    max_articles_to_process = 5000
    max_chunks_to_generate = 20000

    print(f"Processing up to {max_articles_to_process} articles to generate ~{max_chunks_to_generate} text chunks...")

    # Use tqdm for a progress bar on the streamed dataset
    pbar = tqdm(total=max_articles_to_process)
    for i, article in enumerate(wiki_dataset_stream):
        if i >= max_articles_to_process or len(knowledge_base_texts) >= max_chunks_to_generate:
            break
        article_chunks = chunk_text(article['text'])
        knowledge_base_texts.extend(article_chunks)
        pbar.update(1)
    pbar.close()

    print(f"✅ Loaded and chunked {len(knowledge_base_texts)} text sections from Wikipedia.")

    # load the embedding model
    print("\nLoading sentence-transformer model for embeddings...")
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device=DEVICE)
    print("✅ Embedding model loaded.")

    # create vector embedding
    print("\nCreating vector embeddings for the knowledge base (this may take a few minutes)...")
    embeddings = embedding_model.encode(knowledge_base_texts, show_progress_bar=True)
    embeddings = np.array(embeddings).astype('float32')
    print(f"✅ Embeddings created with shape: {embeddings.shape}")

    # Build FAISS index
    print("\nBuilding FAISS index...")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    print(f"✅ FAISS index built. Total entries: {index.ntotal}")

    # save index and knowledge
    print(f"\nSaving FAISS index to: {config.VECTOR_DB_PATH}")
    faiss.write_index(index, config.VECTOR_DB_PATH)

    print(f"Saving knowledge base texts to: {config.KNOWLEDGE_BASE_PATH}")
    with open(config.KNOWLEDGE_BASE_PATH, 'w') as f:
        json.dump(knowledge_base_texts, f)

    print("✅ Vector database and knowledge base saved to Google Drive.")

except Exception as e:
    print(f"❌ An error occurred during vector database creation: {e}")
    print("   Skipping the rest of this cell.")





--- Starting Vector Database Creation ---
Loading official Wikipedia dataset (this may take a moment)...


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Processing up to 5000 articles to generate ~20000 text chunks...


  0%|          | 0/5000 [00:00<?, ?it/s]

✅ Loaded and chunked 20005 text sections from Wikipedia.

Loading sentence-transformer model for embeddings...
✅ Embedding model loaded.

Creating vector embeddings for the knowledge base (this may take a few minutes)...


Batches:   0%|          | 0/626 [00:00<?, ?it/s]

✅ Embeddings created with shape: (20005, 384)

Building FAISS index...
✅ FAISS index built. Total entries: 20005

Saving FAISS index to: /content/drive/MyDrive/SLM_Project/faiss_wikipedia_index.bin
Saving knowledge base texts to: /content/drive/MyDrive/SLM_Project/wikipedia_chunks.json
✅ Vector database and knowledge base saved to Google Drive.


In [ ]:
#interactive RAG chat with vector databse
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

print("\n--- Initializing Advanced RAG Chat System ---")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Shared configuration
class ProjectConfig:
    DRIVE_PROJECT_DIR = "/content/drive/MyDrive/SLM_Project"
    STUDENT_MODEL_OUTPUT_DIR = os.path.join(DRIVE_PROJECT_DIR, "distilgpt2_dolly_direct_fine_tuned")
    VECTOR_DB_PATH = os.path.join(DRIVE_PROJECT_DIR, "faiss_wikipedia_index.bin")
    KNOWLEDGE_BASE_PATH = os.path.join(DRIVE_PROJECT_DIR, "wikipedia_chunks.json")

config = ProjectConfig()

# load data and components
print("Loading all necessary components from Google Drive...")
try:
    chat_tokenizer = AutoTokenizer.from_pretrained(config.STUDENT_MODEL_OUTPUT_DIR, local_files_only=True)
    chat_model = AutoModelForCausalLM.from_pretrained(config.STUDENT_MODEL_OUTPUT_DIR, local_files_only=True).to(DEVICE)
    chat_model.eval()

    embedding_model_chat = SentenceTransformer('all-MiniLM-L6-v2', device=DEVICE)

    index_chat = faiss.read_index(config.VECTOR_DB_PATH)

    with open(config.KNOWLEDGE_BASE_PATH, 'r') as f:
        knowledge_base_chat = json.load(f)

    print("✅ All components loaded successfully.")
except Exception as e:
    print(f"❌ Error loading components: {e}")
    print("Please ensure that CELL 2 (Training) and CELL 3 (Build Vector DB) have been run successfully.")
    exit()


# add semantic search capabilities to the RAG inference function
def get_semantic_rag_response(prompt, model, tokenizer, k=3, max_new_tokens=200):
    print("   > Performing semantic search...")

    truncated_prompt = prompt[:1000]

    query_embedding = embedding_model_chat.encode([truncated_prompt], convert_to_tensor=True).cpu().numpy()
    distances, indices = index_chat.search(query_embedding, k)

    context = ""
    if indices.size > 0:
        retrieved_texts = [knowledge_base_chat[i] for i in indices[0]]
        context = "\n\n---\n\n".join(retrieved_texts)
        print(f"   > Context found from {len(retrieved_texts)} chunks.")
    else:
        print("   > No relevant context found in the vector database.")

    rag_prompt = (
        "Based on the following context, answer the user's question. If the context does not contain the answer, "
        "state that the information is not available in the provided text.\n\n"
        f"Context:\n{context}\n\n"
        f"Instruction:\n{prompt}\n\n"
        "Response:\n"
    )

    model_max_length = model.config.max_position_embeddings
    max_prompt_length = model_max_length - max_new_tokens

    inputs = tokenizer(
        rag_prompt,
        return_tensors="pt",
        max_length=max_prompt_length,
        truncation=True
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs, max_new_tokens=max_new_tokens, temperature=0.7,
            top_p=0.9, do_sample=True, repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )
    response_text = tokenizer.decode(outputs[0, inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return response_text

# The RAG chat loop- start asking questions and get answers from RAG
print("\n--- Interactive Chat with Semantic Search RAG ---")
print("Ask a factual question. The model will now use a vector database to find context.")
print("Type 'quit' or 'exit' to end the chat.")
print("--------------------------------------------------")

while True:
    user_prompt = input("You: ")
    if user_prompt.lower() in ['quit', 'exit']:
        print("Exiting chat. Goodbye!")
        break
    print("Model is thinking...")
    model_response = get_semantic_rag_response(user_prompt, chat_model, chat_tokenizer)
    print(f"Model: {model_response}\n")




--- Initializing Advanced RAG Chat System ---
Loading all necessary components from Google Drive...
✅ All components loaded successfully.

--- Interactive Chat with Semantic Search RAG ---
Ask a factual question. The model will now use a vector database to find context.
Type 'quit' or 'exit' to end the chat.
--------------------------------------------------
You: what is gravity?
Model is thinking...
   > Performing semantic search...
   > Context found from 3 chunks.
Model:  the acceleration of a falling object depends entirely upon its acceleration relative to any other body. When applying two equations, the acceleration of the falling objects causes them both absolute accelerations. Thus one can say that the acceleration with respect to movement is the distance from their origin to the location of rotation. An example of such an acceleration would depend largely about the speed of the falling object.  For example if you travel 2 kilometers per hour, then your velocity will be aroun


KeyboardInterrupt



In [ ]:

# Model evaluation - use different metrics

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import time
import evaluate
from datasets import load_dataset

print("\n--- Initializing Model Evaluation ---")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# shared configuration
class ProjectConfig:
    DRIVE_PROJECT_DIR = "/content/drive/MyDrive/SLM_Project"
    STUDENT_MODEL_OUTPUT_DIR = os.path.join(DRIVE_PROJECT_DIR, "distilgpt2_squad_fine_tuned")

    # evaluation
    EVAL_DATASET_ID = "squad"
    NUM_EVAL_SAMPLES = 100

config = ProjectConfig()

# load model for evaluation
print("Loading fine-tuned model from Google Drive...")
try:
    eval_tokenizer = AutoTokenizer.from_pretrained(config.STUDENT_MODEL_OUTPUT_DIR, local_files_only=True)
    eval_model = AutoModelForCausalLM.from_pretrained(config.STUDENT_MODEL_OUTPUT_DIR, local_files_only=True).to(DEVICE)
    eval_model.eval()
    print(f"✅ Model loaded successfully from '{config.STUDENT_MODEL_OUTPUT_DIR}'.")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit()

# load dataset for validation
# use the val split to ensure the model has never seen this data.
print("\nLoading SQuAD validation set for benchmarking...")
eval_dataset = load_dataset(config.EVAL_DATASET_ID, split='validation')
# Select a subset for faster evaluation
eval_dataset = eval_dataset.select(range(config.NUM_EVAL_SAMPLES))
print(f"✅ Loaded {len(eval_dataset)} samples from the validation set.")

# generate responses for model evalution
print("\nGenerating model responses for evaluation...")
model_generations = []
ground_truth_answers = []

# This prompt template must EXACTLY match the one used for training
rag_template = "Context:\n{context}\n\nInstruction:\n{question}\n\nResponse:\n"

for item in eval_dataset:
    prompt_text = rag_template.format(
        context=item["context"],
        question=item["question"],
        answer=""
    )

    # store the ground truth answer for comparison
    ground_truth_answers.append(item["answers"]["text"][0])

    inputs = eval_tokenizer(
        prompt_text,
        return_tensors="pt",
        max_length=eval_model.config.max_position_embeddings - 150,
        truncation=True
    ).to(DEVICE)

    with torch.no_grad():
        outputs = eval_model.generate(**inputs, max_new_tokens=100, pad_token_id=eval_tokenizer.eos_token_id)

    response_text = eval_tokenizer.decode(outputs[0, inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    model_generations.append(response_text)

print("✅ Model responses generated.")

# calculate text quality matrics
print("\nCalculating text quality metrics (ROUGE, BLEU, BERTScore)...")
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')
bertscore = evaluate.load("bertscore")

rouge_scores = rouge.compute(predictions=model_generations, references=ground_truth_answers)
bleu_scores = bleu.compute(predictions=model_generations, references=ground_truth_answers)
bert_scores = bertscore.compute(predictions=model_generations, references=ground_truth_answers, lang="en")
avg_bert_score_f1 = np.mean(bert_scores['f1'])
print("✅ Text quality metrics calculated.")

# Calculate efficiency metrics
print("\nCalculating efficiency metrics...")
model_params = eval_model.num_parameters()

inference_times = []

_ = eval_model.generate(**inputs, max_new_tokens=10)

for item in eval_dataset:
    prompt_text = rag_template.format(context=item["context"], question=item["question"], answer="")
    inputs = eval_tokenizer(prompt_text, return_tensors="pt", max_length=512, truncation=True).to(DEVICE)

    start_time = time.perf_counter()
    with torch.no_grad():
        eval_model.generate(**inputs, max_new_tokens=100, pad_token_id=eval_tokenizer.eos_token_id)
    torch.cuda.synchronize()
    end_time = time.perf_counter()
    inference_times.append(end_time - start_time)

avg_inference_time = np.mean(inference_times)
print("✅ Efficiency metrics calculated.")
#display benchmark results
print("\n\n========================= MODEL BENCHMARK SUMMARY =========================")
print(f"Evaluation against {config.NUM_EVAL_SAMPLES} samples from the SQuAD validation set.")
print("-" * 75)
print("--- Text Quality (Higher is Better) ---")
print(f"{'ROUGE-L Score':<28} | {rouge_scores['rougeL']:.4f}")
print(f"{'BLEU Score':<28} | {bleu_scores['bleu']:.4f}")
print(f"{'BERTScore (F1)':<28} | {avg_bert_score_f1:.4f}")
print("-" * 75)
print("--- Efficiency ---")
print(f"{'Parameter Count':<28} | {model_params/1e6:.1f} M")
print(f"{'Avg. Inference Time (s)':<28} | {avg_inference_time:.4f} s")
print("===========================================================================")



--- Initializing Model Evaluation ---
Loading fine-tuned model from Google Drive...
✅ Model loaded successfully from '/content/drive/MyDrive/SLM_Project/distilgpt2_squad_fine_tuned'.

Loading SQuAD validation set for benchmarking...
✅ Loaded 100 samples from the validation set.

Generating model responses for evaluation...
✅ Model responses generated.

Calculating text quality metrics (ROUGE, BLEU, BERTScore)...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Text quality metrics calculated.

Calculating efficiency metrics...
✅ Efficiency metrics calculated.


========================= MODEL BENCHMARK SUMMARY =========================
Evaluation against 100 samples from the SQuAD validation set.
---------------------------------------------------------------------------
--- Text Quality (Higher is Better) ---
ROUGE-L Score                | 0.0374
BLEU Score                   | 0.0041
BERTScore (F1)               | 0.7993
---------------------------------------------------------------------------
--- Efficiency ---
Parameter Count              | 81.9 M
Avg. Inference Time (s)      | 0.5638 s


In [ ]:
#standard benchmarks

import os
import json


#Install LM evaluation harness
print("Installing LM Evaluation Harness and its dependencies...")
# We install from the git repository to ensure we have the latest version
!pip install -q git+https://github.com/EleutherAI/lm-evaluation-harness.git

print("✅ Evaluation harness installed.")


#configuration
# Define the paths to your model and where to save the results.
class BenchmarkingConfig:
    DRIVE_PROJECT_DIR = "/content/drive/MyDrive/SLM_Project"

    MODEL_PATH = os.path.join(DRIVE_PROJECT_DIR, "distilgpt2_squad_fine_tuned")

    RESULTS_PATH = os.path.join(DRIVE_PROJECT_DIR, "benchmark_results.json")


    TASKS = "hellaswag,truthfulqa,mmlu,humaneval"

config = BenchmarkingConfig()

print(f"\nModel to be evaluated: {config.MODEL_PATH}")
print(f"Benchmarks to run: {config.TASKS}")


#Run the Evaluation ---
os.environ["HF_ALLOW_CODE_EVAL"] = "1"
print("\n'HF_ALLOW_CODE_EVAL' environment variable set to '1'.")


print("\n--- Starting Benchmark Evaluation (this can take a very long time, especially MMLU) ---")

!lm_eval --model hf \
    --model_args pretrained={config.MODEL_PATH},trust_remote_code=True \
    --tasks {config.TASKS} \
    --device cuda:0 \
    --batch_size 8 \
    --output_path {config.RESULTS_PATH}

print("\n--- Benchmark Evaluation Complete ---")


# display a Summary of the Results

print("\n--- Benchmark Results Summary ---")

try:
    with open(config.RESULTS_PATH, 'r') as f:
        results = json.load(f)

    print(f"\nResults saved to: {config.RESULTS_PATH}")
    print("=" * 50)

    for task_name, task_data in results["results"].items():
        print(f"\nBenchmark: {task_name}")
        for metric, value in task_data.items():

            if 'acc' in metric or 'hendrycksTest' in metric or 'pass@' in metric:
                 print(f"  - {metric}: {value:.4f}")

    print("\n*Note: 'hendrycksTest-*-acc' is the metric for MMLU sub-tasks.")
    print("*Note: 'pass@k' is the metric for HumanEval.")
    print("=" * 50)

except FileNotFoundError:
    print(f"❌ Error: Results file not found at '{config.RESULTS_PATH}'. The evaluation may have failed.")
except Exception as e:
    print(f"An error occurred while parsing the results: {e}")



Installing LM Evaluation Harness and its dependencies...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✅ Evaluation harness installed.

Model to be evaluated: /content/drive/MyDrive/SLM_Project/distilgpt2_squad_fine_tuned
Benchmarks to run: hellaswag,truthfulqa,mmlu,humaneval

'HF_ALLOW_CODE_EVAL' environment variable set to '1'.

--- Starting Benchmark Evaluation (this can take a very long time, especially MMLU) ---
2025-09-24 15:36:00.048339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758728160.069682   79506 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758728160.076177   79506 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to